In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from langdetect import detect
import os
import re
import liste_muvra
from lxml import etree
import bs4 as bs
import glob
import shutil

### Listage des fichiers TXT et transformation en fichiers XML

In [2]:
parent = os.getcwd()

In [3]:
list_files = glob.glob("./text_section/*.txt")

In [4]:
for file in list_files:
    with open(file, 'r') as f :
        t = f.readlines()
    t = ''.join(t)
    t = re.sub(r"^\n", r"", t)
    t = re.sub(r"-\n", r"", t)
    t = re.sub(r"\n", r" ", t)
    t = re.sub(r" <section> ", r"\n", t)
    t = re.sub(r"<section> ", r"", t)
    with open(file, 'w') as f :
        f.write(t)

In [5]:
list_aut_str = liste_muvra.auteurs_muvra
list_regiolect = liste_muvra.regiolectes
for file in list_files:
    with open(file, "r") as f:
        text = f.readlines()
    print(file)
    lang = []
    for i in text:
        l = detect(i)
        if l == 'ro' or l == 'ca':
            lang.append('co')
        elif l == 'it':
            test = i.split()
            count_u = test.count('u')
            count_il = test.count('il')
            if count_il > count_u:
                lang.append(l)
            else:
                lang.append('co')
        else:
            lang.append(l)
    
    
    aut = []
    for t in text:
        t = re.sub(r"’", r"'",t)
        dic = {}
        for i in list_aut_str:
            if i in t[-70:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            aut.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            aut.append('Inconnu')

    typ = []
    list_rub = ['art','let','poe','div','the','cha','feu','his','trd']
    for t in text:
        dic = {}
        for i in list_rub:
            if i in t[-5:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            typ.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            typ.append('Non défini')

    reg = []
    for r in text:
        dic = {}
        for i in list_regiolect:
            if i in r[-100:]:
                dic[i] = 1
            else:
                dic[i] = 0
        if 1 in dic.values():
            reg.append(list(dic.keys())[list(dic.values()).index(1)])
        else:
            reg.append('Inconnu')
            
    df = pd.DataFrame([text,lang,aut,typ,reg]).T
    df.columns = ['texte','langue','auteur','type','regiolecte']
    xml = df.to_xml()
    myre = '(bpt\w{7,10})'
    if re.search(myre, file):
        nfile = re.search(myre, file).group(1)
    with open(nfile + '.xml', "w+") as f:
        f.write(xml)

./text_section/bpt6k13309974.txt
./text_section/bpt6k13309744.txt
./text_section/bpt6k13309514.txt
./text_section/bpt6k13309818.txt
./text_section/bpt6k13309655.txt
./text_section/bpt6k13309351.txt
./text_section/bpt6k13309247.txt
./text_section/bpt6k13309959.txt
./text_section/bpt6k13309670.txt
./text_section/bpt6k13309566.txt


### Modification des balises XML pour qu'elles correspondent à nos standards

In [6]:
list_xml_files = glob.glob("*.xml")

In [7]:
for file in list_xml_files:
    xml_file = ET.parse(file)
    rootElement = xml_file.getroot()
    for element in rootElement.iter('data'):
        element.tag = 'revue'
    for element in rootElement.iter('row'):
        element.tag = 'rubrique'
    item = rootElement.find("rubrique")
    for element in item.iter('rubrique'):
        element.tag = 'manchette'
    xml_file.write(file)


In [8]:
list_files

['./text_section/bpt6k13309974.txt',
 './text_section/bpt6k13309744.txt',
 './text_section/bpt6k13309514.txt',
 './text_section/bpt6k13309818.txt',
 './text_section/bpt6k13309655.txt',
 './text_section/bpt6k13309351.txt',
 './text_section/bpt6k13309247.txt',
 './text_section/bpt6k13309959.txt',
 './text_section/bpt6k13309670.txt',
 './text_section/bpt6k13309566.txt']

In [9]:
for file in list_xml_files: 
    with open(file, 'r') as f:
        content = f.read()
    soup = bs.BeautifulSoup(content,'xml')
    for tag in soup.find_all("langue"):
        tag.parent['lan'] = tag.get_text()
    for tag in soup.find_all("type"):
        tag.parent['typ'] = tag.get_text()
    for tag in soup.find_all("regiolecte"):
        cont = tag.get_text()
        if re.search("Parlata di a Custera",cont):
            tag.parent['regio'] = "custera"
        elif re.search("Parlata balanina",cont):
            tag.parent['regio'] = "balagna"
        elif re.search("Parlatu di a Castagniccia",cont):
            tag.parent['regio'] = "castagniccia"
        elif re.search("Parlata di Sartè",cont):
            tag.parent['regio'] = "sarte"
        elif re.search("Parlata di Levie",cont):
            tag.parent['regio'] = "levie"
        elif re.search("Parlata di Petretu",cont):
            tag.parent['regio'] = "petretu"
        elif re.search("Dialettu aiaccinu",cont):
            tag.parent['regio'] = "aiaccinu"
        else:
            tag.parent['regio'] = "inc"
    with open(file, 'w') as f:
        f.write(soup.prettify())

In [10]:
list_rub = ['art','let','poe','div','the','cha','feu','his','trd']
for file in list_xml_files:
    with open(file, 'r') as f:
        content = f.read()
    soup = bs.BeautifulSoup(content,'xml')
    for tag in soup.find_all("texte"):
        t = tag.get_text()
        for i in list_rub:
            if i in t[-7:]:
                t = re.sub(" " + i + r'\n','', t)
        for i in list_aut_str:
            if i in t[-30:]:
                t = re.sub(" " + i,'', t)            
        tag.string.replace_with(t)
    with open(file, 'w') as f:
        f.write(soup.prettify())

-----

### Vérifions que les ficheirs XML traités sont valides avant de les déplacer

In [11]:
def parsing_xml_corsican_press(file):
    '''
    Fonction permettant de parser des XML (avec DTD spécifique), pour retourner des listes de chaque élément.
    
    Entrée : XML
    Sortie : listes
    '''

    types = []
    langue = []
    texte = []
    auteur = []
    fichier = []
    regiolecte = []

    tree = etree.parse(file)
    
    for typ in tree.xpath("//type/text()"):
        typ = re.sub(r"\n ", "", typ)
        typ = re.sub(r"\n", "", typ)
        typ = re.sub(r"  ", "", typ)
        types.append(typ)
        fichier.append(file)

    for lan in tree.xpath("//langue/text()"):
        lan = re.sub(r"\n ", "", lan)
        lan = re.sub(r"\n", "", lan)
        lan = re.sub(r"  ", "", lan)
        langue.append(lan)

    for tex in tree.xpath("//texte/text()"):
        tex = re.sub(r"\n ", "", tex)
        tex = re.sub(r"\n", "", tex)
        texte.append(tex)

    for aut in tree.xpath("//auteur/text()"):
        aut = re.sub(r"\n ", "", aut)
        aut = re.sub(r"\n", "", aut)
        aut = re.sub(r"  ", "", aut)
        auteur.append(aut)

    for reg in tree.xpath("//regiolecte/text()"):
        reg = re.sub(r"\n ", "", reg)
        reg = re.sub(r"\n", "", reg)
        reg = re.sub(r"  ", "", reg)
        regiolecte.append(reg)
    
    
    
    return auteur, types, langue, texte, regiolecte, fichier

In [12]:
all_author = []
all_types = []
all_lang = []
all_text = []
all_regio = []
all_file = []

for file in list_xml_files:
    
    print(file)
    author, typo, lang, text, regiolecte, fichier = parsing_xml_corsican_press(file)
    all_author = all_author + author
    all_types = all_types + typo
    all_lang = all_lang + lang
    all_text = all_text + text
    all_regio = all_regio + regiolecte
    all_file = all_file + fichier

bpt6k13309351.xml
bpt6k13309566.xml
bpt6k13309670.xml
bpt6k13309744.xml
bpt6k13309959.xml
bpt6k13309655.xml
bpt6k13309247.xml
bpt6k13309514.xml
bpt6k13309818.xml
bpt6k13309974.xml


In [13]:
len(all_regio)

138

In [14]:
data = {"Auteur":all_author, "Type":all_types, "Langue":all_lang, "Régiolecte":all_regio, "Texte":all_text, "Fichier":all_file}

#data['Titre'].append(titre)
#data['Auteur'].append(auteur)
#data['Type'].append(types)
#data['Langue'].append(langue)
#data['Texte'].append(texte)

df_data = pd.DataFrame.from_dict(data)
df_data.to_csv("test_donnees.csv")

In [15]:
df_data

,Auteur,Type,Langue,Régiolecte,Texte,Fichier
0,Inconnu,div,co,Inconnu,X1 — Numaru 38; _ i Annu ? Esce a Dumenica D...,bpt6k13309351.xml
1,P. di B.,art,co,Inconnu,"Segni di Schiavitù “C’est un Corsico!"" E d...",bpt6k13309351.xml
2,Simonu di San Jorghiu,poe,co,Inconnu,"Dopu d'avè invucatu în u so' cori, Cun un s...",bpt6k13309351.xml
3,Ghiacintu di Muru,art,co,Inconnu,Parolle di Fede Semu Noi... Un pudemu nasc...,bpt6k13309351.xml
4,Inconnu,div,fr,Inconnu,Opinioni sopra a Corsica e i Corsi Entre h...,bpt6k13309351.xml
...,...,...,...,...,...,...
133,Saveriu Malaspina,art,co,Inconnu,A nostra Santa Fede (Catechismu corsu) (Se...,bpt6k13309974.xml
134,Ghiuvanni Olivieri,poe,co,Inconnu,"A L'ORSU D'OREZZA D'a valle di Piedicroce, ...",bpt6k13309974.xml
135,Kyrn,art,fr,Inconnu,L’Affaire d’Alata JUSTICE! Dans un tout ré...,bpt6k13309974.xml
136,Polverinu,poe,co,Inconnu,"Nutizie religiose U nostru amicu, u sgiò ab...",bpt6k13309974.xml


### Classons les fichiers

In [16]:
#for file in glob.glob("*.xml"):
#    shutil.move(file, parent + "/xml/")